In [1]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import config
from derma.dataset import Derma
from derma.architecture import InvertedResidual

import numpy as np
import torch
from torch.utils.data import WeightedRandomSampler, DataLoader
from torchvision.models import MobileNetV2
from torchvision import transforms

torch.cuda.is_available()

True

# Problem parameters

In [2]:
# DATASET
labels = [0, 1]
# labels = [1, 2, 3, 4] # No funciona. Check criterion = nn.CrossEntropyLoss()

dataset_dir = os.path.join(config.DATASET_DIR,'0128-0128_dullrazor')
problem_name = 'Mn_irv00_2_128-dullrazor_weighted' 
# problem_name = 'block_setting_classes_size-hairs_weighted-classes'

split_ratio = 0.9 # x train, 1-x test
batch_size = 32
transform = transforms.Compose([
    transforms.ToTensor(),
#    transforms.Resize((128, 128))
])

# MODEL
# Original setting for mobilenet v2 (https://github.com/pytorch/vision/blob/main/torchvision/models/mobilenetv2.py)
inverted_residual_setting = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ]

CoordAtt = False
criterion = torch.nn.CrossEntropyLoss()

# TRAINING
n_epoch = 10
Weighted_sampling = True
log_dir = os.path.join('log',problem_name) 
model_dir = os.path.join('models',problem_name+'.pt')

# Dataset

In [3]:
# Derma dataset 
dataset = Derma(dataset_dir,labels=labels,transform=transform)

# Train-test splitting
#dataset.shuffle(manual_seed=42) 
train_set, test_set = dataset.split_rand(split_ratio=0.9,manual_seed=42)

# Weighted sampling
if Weighted_sampling:
    from derma.dataset import get_samples_weight
    samples_weight = get_samples_weight(train_set)
    sampler = WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)

    # Data loaders
    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=0, sampler=sampler)
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=0, sampler=sampler)
else:
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=0)


#dataset = datasets.CIFAR100('data', train=True, transform=transform, download=True)
#loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Model

In [4]:
# ISIC2018
if CoordAtt:
    model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting, block=InvertedResidual)
else:
    model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting) # standard MobileNetV2

#model = MobileNetV2(num_classes=100, inverted_residual_setting=inverted_residual_setting, block=InvertedResidual) # CIFAR
#input_test = torch.rand((1,3,64,64))
#print('Features Output shape: {}'.format(model.features(input_test).shape))
#print('Classifier Output shape: {}'.format(model(input_test).shape))

# Training

In [5]:
from derma.utils import train
from torch.utils.tensorboard import SummaryWriter

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
tb_writer = SummaryWriter(log_dir=log_dir)

train(model, train_loader, optimizer, criterion, n_epoch, tb_writer)

100%|██████████| 10/10 [41:30<00:00, 249.06s/epoch, acc=0.9881, tls=0.0332]


In [6]:
torch.save(model.state_dict(), model_dir)

# Test

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    model.load_state_dict(torch.load(model_dir))
    model.to(device)
    model.eval()
else:
    model.load_state_dict(torch.load(model_dir), map_location=torch.device('cpu'))
    model.eval()